In [2]:
import asyncio
from pyppeteer import launch
import csv
import pandas as pd
import re

In [3]:
job_df=None

In [7]:
####test fields
async def scrape_indeed():
    global job_df
    global job_listings
    browser = await launch(headless=False)
    page = await browser.newPage()


    await page.goto('https://www.indeed.com')

    # Wait for 5 seconds
    await asyncio.sleep(5) 
    # Input job title
    await page.type('input[name="q"]', "data analyst")

     # Clear the default location
    location_input_selector = 'input[name="l"]'
    await page.click(location_input_selector, {'clickCount': 3})  # Triple click to select all text
    await page.keyboard.press('Backspace')  # Clear the text

    # Input new location
    await page.type(location_input_selector, "USA")

    # Click search button
    await page.click('button[type="submit"]')
    # Wait for 5 seconds
    await asyncio.sleep(5) 


    # Apply "Date Posted" filter for "Last 3 days"
    await page.waitForSelector('#filter-dateposted', {'visible': True})
    await page.click('#filter-dateposted')

    # Wait for the dropdown to become visible and select the "Last 3 days" option
    await page.waitForSelector('a[href*="&fromage=3"]', {'visible': True})
    await page.click('a[href*="&fromage=3"]')


    # Max go to 10 pages
    num_pages=10
    job_info={
        'title':[],
        'company':[],
        'location':[],
        'post_date': [],
        'rating':[],
        'salary':[],
        'job_link':[],
        'jd':[]
    }

    for pg in range(num_pages):


            

        # Wait for 5 seconds
        await asyncio.sleep(5) 

        job_listings = await page.querySelectorAll('.job_seen_beacon')
        
        for job in job_listings:
            # Extract the job title
            try:
                title_element = await job.querySelector('h2.jobTitle span[title]')
                title = await page.evaluate('(element) => element.textContent', title_element)
            except Exception as e:
                title="Not Found"


            # Extract the company name
            try:
                company_element = await job.querySelector('div.company_location [data-testid="company-name"]')
                company = await page.evaluate('(element) => element.textContent', company_element)
            except Exception as e:
                company="Not Found"

            # Extract jd
            try:
                jd_element = await job.querySelector('div.underShelfFooter div ul')
                jd = await page.evaluate('(element) => element.textContent', jd_element)
            except Exception as e:
                jd="Not Found"

            # Get job link
            try:
                job_element = await job.querySelector('h2.jobTitle a')
                job_link = await page.evaluate('(element) => element.getAttribute("href")', job_element)
            except Exception as e:
                job_link="Not Found"


            # Extract the rating
            try:
                rating_element = await job.querySelector('div.company_location span [data-testid="holistic-rating"]')
                rating = await page.evaluate('(element) => element.textContent', rating_element)
            except Exception as e:
                rating="Not Found"

            # Extract the date
            try:
                date_element = await job.querySelector('div.underShelfFooter [data-testid="myJobsStateDate"]')
                post_date = await page.evaluate('(element) => element.textContent', date_element)
            except Exception as e:
                post_date="Not Found"

            # Extract the salary
            try:
                salary_element = await job.querySelector('div.jobMetaDataGroup [data-testid="attribute_snippet_testid"]')
                salary = await page.evaluate('(element) => element.textContent', salary_element)
            except Exception as e:
                salary="Not Found"

            # Extract the location
            location_element = await job.querySelector('div.company_location [data-testid="text-location"]')
            location = await page.evaluate('(element) => element.textContent', location_element)

            job_info['title'].append(title)
            job_info['company'].append(company)
            job_info['location'].append(location)
            job_info['post_date'].append(post_date)
            job_info['rating'].append(rating)
            job_info['salary'].append(salary)
            job_info['job_link'].append(job_link)
            job_info['jd'].append(jd)

            #click for next page
        try:
            await page.waitForSelector('nav [data-testid="pagination-page-next"]')
            await page.click('nav [data-testid="pagination-page-next"]')
        except Exception as e:
            print("no more page")
            break
        
        

    job_df=pd.DataFrame.from_dict(job_info)

    await browser.close()

In [5]:
await scrape_indeed()

no more page


In [6]:
job_df

,title,company,location,post_date,rating,salary,job_link,jd
0,"Department Data Analyst, HRPP",UT Southwestern,"Dallas, TX 75390 (Design District area)",PostedJust posted,Not Found,Full-time,/rc/clk?jk=e12fd18049322537&bb=wSpuRhSVJQiiMky...,"\n Abstracts data, reconciles data inconsiste..."
1,Data Infrastructure Analyst,"7-Eleven, Inc.","Irving, TX 75063 (Valley Ranch area)",PostedPosted 3 days ago,Not Found,Full-time,/rc/clk?jk=2afdda60528682db&bb=wSpuRhSVJQiiMky...,\n Manage processes and improve data set-up i...
2,"Sr Analyst, Inflight Analytics",American Airlines,"Dallas, TX 75219 (Downtown area)",PostedPosted 1 day ago,Not Found,Not Found,/rc/clk?jk=f9e9ef65e8e797ed&bb=wSpuRhSVJQiiMky...,\n Strong analytical background involving dat...
3,Business Analyst,McKesson,"Irving, TX 75039 (Freeport/Hackberry area)",PostedJust posted,Not Found,"$59,700 - $99,500 a year",/rc/clk?jk=36baed663181cb0f&bb=wSpuRhSVJQiiMky...,\n 2 + years business analyst experience.\n E...
4,Business Support Analyst Lead – Workforce Plan...,USAA,"Hybrid work in Plano, TX 75023",PostedJust posted,3.7,Not Found,/pagead/clk?mo=r&ad=-6NYlbfkN0CdTBpsLrhs4IwmIs...,"\n Gathers information, analyzes data and tre..."
5,Senior Data & Insights Analyst- 24010451,MoneyGram,"Remote in Dallas, TX",PostedJust posted,Not Found,Not Found,/rc/clk?jk=6b0136b7986d482e&bb=wSpuRhSVJQiiMky...,\n Proficient in SQL to query large data plat...
6,"Business Analyst, Payments/Marketplace",Wellfit Technologies Inc.,"Hybrid work in Irving, TX 75039",PostedJust posted,Not Found,"From $120,000 a year",/rc/clk?jk=db0613b310994fee&bb=wSpuRhSVJQiiMky...,\n Full-cycle business analyst lifecycle: 5 y...
7,CRM Data Analysts – Lead,InfosysLimited,"Dallas, TX 75202 (Downtown area)",PostedJust posted,Not Found,"$90,000 - $100,000 a year",/rc/clk?jk=b894b55a10273b0a&bb=wSpuRhSVJQiiMky...,"\n Expert experience with data ETL process, d..."
8,Business Analyst (Retail Banking),Wipro Limited,"Irving, TX",PostedJust posted,Not Found,Not Found,/rc/clk?jk=95a502b8e88d52e6&bb=wSpuRhSVJQiiMky...,\n You are expected to elicit and detail the ...
9,Business Intelligence Consultant,Verizon,"Irving, TX",PostedJust posted,Not Found,Full-time,/rc/clk?jk=e31e0b4aec962639&bb=wSpuRhSVJQiiMky...,\n You’ll be analyzing data and creating data...


In [181]:
#function to parse salary range
def salary_limit(salary):
    matches=re.findall(r'\$([\d,]+(\.\d+)?)',salary)
    first = matches[0][0] if len(matches)>0 else None
    second = matches[1][0] if len(matches)>1 else None
    return first,second
#function to parse salary pay period
def salary_period(salary):
    if salary.endswith('hour'):
        return 'hourly'
    elif salary.endswith('month'):
        return 'monthly'
    elif salary.endswith('year'):
        return 'yearly'


In [182]:
job_df['pay_period']=job_df['salary'].apply(salary_period)

In [178]:
job_df[['salary_first','salary_second']]=job_df['salary'].apply(lambda x: pd.Series(salary_limit(x)))

In [179]:
job_df['salary_first']=job_df['salary_first'].str.replace(',','').astype(float)
job_df['salary_second']=job_df['salary_second'].str.replace(',','').astype(float)

In [183]:
job_df

,title,company,location,post_date,rating,salary,job_link,jd,salary_list,"(salary_first, salary_second)",salary_first,salary_second,pay_period
0,Crime Data Analyst,City of Bloomington,"Bloomington, IL 61701",PostedJust posted,Not Found,$31.66 an hour,/rc/clk?jk=54f9a6802b75d1d3&bb=iZxmYlc3Pbh3sfH...,\n Develop data sampling techniques and selec...,[31.66],"(31.66, None)",31.66,NaN,hourly
1,Data Analyst II,Health & Human Services Comm,"Austin, TX 78728",PostedJust posted,Not Found,"$4,013.16 - $5,234.75 a month",/rc/clk?jk=162f1c80f65daf08&bb=iZxmYlc3Pbh3sfH...,\n Management of large volumes of data.\n Exp...,"[4,013.16, 5,234.75]","(4,013.16, 5,234.75)",4013.16,5234.75,monthly
2,Data Analyst,Service Professor,"Grand Rapids, MI 49512",PostedJust posted,Not Found,"$70,000 - $80,000 a year",/rc/clk?jk=b26fba0c99cae49b&bb=iZxmYlc3Pbh3sfH...,\n Data Governance: Knowledge of data quality...,"[70,000, 80,000]","(70,000, 80,000)",70000.00,80000.00,yearly
3,Research Data Analyst,"University of California, Irvine","Remote in Irvine, CA 92697",PostedJust posted,Not Found,Full-time,/rc/clk?jk=d73b7c9cd4989597&bb=iZxmYlc3Pbh3sfH...,\n Working skills in geospatial data on addre...,[],"(None, None)",NaN,NaN,None
4,Master Data Analyst,Milwaukee Electric Tool Corporation,"Milwaukee, WI",PostedJust posted,Not Found,Full-time,/rc/clk?jk=815de30f297b4c09&bb=tBjcC9UyaqBbQCb...,\n Review audit reports to ensure data accura...,[],"(None, None)",NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,Board of Elections Business Analyst,Wake County Government,"Raleigh, NC 27610 (Southeast area)",PostedJust posted,Not Found,"$64,971 - $87,712 a year",/rc/clk?jk=103eb4506e3843d1&bb=sjA3YeuVOrNcUPM...,\n Work with a contracted vendor to prepare E...,"[64,971, 87,712]","(64,971, 87,712)",64971.00,87712.00,yearly
135,Business Analyst - Deliverables - Remote,Optum,"Remote in Washington, DC 20001",PostedPosted 2 days ago,Not Found,Not Found,/rc/clk?jk=6c63555505277911&bb=sjA3YeuVOrNcUPM...,\n Demonstrated understanding of the 5 stages...,[],"(None, None)",NaN,NaN,None
136,CRM Business Analyst,Milton CAT,"Milford, MA 01757",PostedPosted 2 days ago,Not Found,Full-time,/rc/clk?jk=36fe6c4235b6bdbd&bb=sjA3YeuVOrNcUPM...,\n Experience in data analytics and data vali...,[],"(None, None)",NaN,NaN,None
137,Data-Metrics Analyst,"Tech-Marine Business, Inc.","Washington, DC",PostedPosted 2 days ago,Not Found,Full-time,/rc/clk?jk=a506514c5553b813&bb=sjA3YeuVOrNcUPM...,\n Ensure data analytics efforts align with o...,[],"(None, None)",NaN,NaN,None


In [154]:
df['Amounts'] = df['Description'].apply(salary_limit)


In [155]:
df['A2']=df['Amounts'].apply()

,Description,Amounts
0,Total cost is $123.45 and the tax is $12.34,"[123.45, 12.34]"
1,Amount is $67.89 with a discount of $5.00,"[67.89, 5.00]"
2,"Price: $10.99, fee $2.00, and additional cost ...","[10.99, 2.00, 3.50]"
3,Fee $5.00,[5.00]
4,No amount here,[]


In [162]:
df['Amounts']

TypeError: cannot do slice indexing on RangeIndex with these indexers [[0]] of type list